## Library to extract Core Statistics from IFS PSO (Planning and Schedule Optimization) XML Schedule
in the current woorkbooks these are mainly used for Comparing WISE Statistics can be also used just a simple summary from any PSO Schedule. For such cases the PowerBI reports from Alex Heinze are definitely a better solution. It can obviously easily be expanded according to everyone needs

Load libraries and xml extractions functions

In [1]:
%run "00 - PSO Data Load Basic lib.ipynb"

## Extract the PSO Statistics as from the &lt;plan&gt; record within the PSO Schedule
These are pre-calculated stats from PSO

In [2]:
def stats_from_plan_data(dataframes_dict):

    #Get the plan data
    df_plan = dataframes_dict['df_plan']
    df_resources = dataframes_dict['df_resources']

    #get the value of out of the stats from the first row of df_plan
    total_allocations = df_plan.iloc[0]['total_allocations']
    total_travel_time = isodate.parse_duration(df_plan.iloc[0]['total_travel_time'])
    total_travel_distance = df_plan.iloc[0]['total_travel_distance']
    average_travel_time = isodate.parse_duration(df_plan.iloc[0]['average_travel_time'])
    average_travel_distance = df_plan.iloc[0]['average_travel_distance'] 
    total_on_site_time = isodate.parse_duration(df_plan.iloc[0]['total_on_site_time'])
    total_private_time = isodate.parse_duration(df_plan.iloc[0]['total_private_time'])
    total_unutilised_time = isodate.parse_duration(df_plan.iloc[0]['total_unutilised_time'])
    schedule_from = df_plan.iloc[0]['schedule_from']
    schedule_to = df_plan.iloc[0]['schedule_to']
    plan_margin = df_plan.iloc[0]['plan_margin']

    #Get data on available resources
    available_resources = df_resources.shape[0]

    # Travel distance per resources
    travel_distance_per_resource = float(total_travel_distance) / int(available_resources)

    #combine all variables above in a single record of a list with name of the variables
    stats_record = {
        'from': schedule_from,
        'to': schedule_to,
        'Margin': plan_margin,
        'tot allocations': total_allocations,
        'Available resources': available_resources,
        'tot travel time': total_travel_time,
        'tot traveldistance': total_travel_distance,
        'avg travel time': average_travel_time,
        'avg traveldistance': average_travel_distance,
        'travel distance per resource': travel_distance_per_resource,
        'tot on site time': total_on_site_time,
        'tot private time': total_private_time,
        'tot unutilised time': total_unutilised_time
    }
    stats = [stats_record]

    return stats


## Calculate some core Stats from the PSO Schedule
These are re-calculated stats from the schedule data

In [ ]:
def stats(dataframes_dict):

    #Get the plan data from alla dataframes
    df_resources = dataframes_dict['df_resources']
    df_allocation_data = dataframes_dict['df_allocation_data']
    df_plan_travel = dataframes_dict['df_plan_travel']
    df_shift = dataframes_dict['df_shift']
    df_shift_break = dataframes_dict['df_shift_break']

    ####################### Calculate Utilization ######################## 
    #calculate total work hours summing df_allocation_data['duration']
    total_work_hours = df_allocation_data['duration'].apply(isodate.parse_duration).sum() 

    #calculate total travel hours summing df_plan_travel['expected_travel_time'] after convertingto isodate
    total_travel_hours = df_plan_travel['expected_travel_time'].apply(isodate.parse_duration).sum()

    #calculate total shift as subtraction of start and end time of shift from df_shift
    total_shift = (df_shift['end_time'] - df_shift['start_time']).apply(isodate.parse_duration).sum()

    #calculate total shift break as sum of df_shift_break['duration']
    total_shift_break = df_shift_break['duration'].apply(isodate.parse_duration).sum()

    #calculate total utlization
    total_utilization = (total_work_hours + total_travel_hours) / (total_shift - total_shift_break)

    #create a stats_record adding utilization
    stats_record = {
        'Utilization': total_utilization
    }
    
    stats = [stats_record]

    return stats


## Wrappers for the two functions above

In [ ]:

def calculate_stats_from_plan(file_to_parse: str):
    
    #Get the plan data
    df_plan = parse_records_to_dataframe_by_file(file_to_parse,'Plan')
    df_resources = parse_records_to_dataframe_by_file(file_to_parse,'Resources')


    #add dataframes to a list of dataframes
    dataframes_dict = {
        'df_plan': df_plan,
        'df_resources': df_resources
    }
    
    return stats_from_plan_data(dataframes_dict)

In [ ]:
import random

def calculate_stats(file_to_parse: str):
    
    #Get the plan data
    df_plan = parse_records_to_dataframe_by_file(file_to_parse,'Plan')
    df_resources = parse_records_to_dataframe_by_file(file_to_parse,'Resources')
    df_allocation_data = parse_records_to_dataframe_by_file(file_to_parse,'Allocation_Data')
    df_plan_travel = parse_records_to_dataframe_by_file(file_to_parse,'Plan_Travel')
    df_shift = parse_records_to_dataframe_by_file(file_to_parse,'Shift')
    df_shift_break = parse_records_to_dataframe_by_file(file_to_parse,'Shift_Break')

    #add dataframes to a list of dataframes
    dataframes_dict = {
        'df_plan': df_plan,
        'df_resources': df_resources,
        'df_allocation_data': df_allocation_data,
        'df_plan_travel': df_plan_travel,
        'df_shift': df_shift,
        'df_shift_break': df_shift_break
    }
    
    return stats_from_plan_data(dataframes_dict)